<a href="https://colab.research.google.com/github/ayyucedemirbas/dinov2_custom_classification/blob/main/dinov2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/facebookresearch/dinov2.git

Cloning into 'dinov2'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 485 (delta 172), reused 160 (delta 160), pack-reused 225
Receiving objects: 100% (485/485), 1.11 MiB | 21.42 MiB/s, done.
Resolving deltas: 100% (240/240), done.


In [2]:
%cd dinov2

/content/dinov2


In [3]:
from dinov2.models.vision_transformer import vit_small, vit_base, vit_large, vit_giant2

/content/dinov2/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/content/dinov2/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/content/dinov2/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [5]:
# A simple notebook demonstrating how to fine-tune a DinoV2 classifier on your own images/labels

# Most of the core code was originally published in an excellent tutorial by Kili Technology, here:
#  https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/main/recipes/finetuning_dinov2.ipynb

# November 11th, 2023 by Lance Legel (lance@3co.ai) from 3co, Inc. (https://3co.ai)

import os
import random
import zipfile
from copy import deepcopy
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torch import nn, optim
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from torchvision import datasets, transforms


In [4]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"username","key":"key"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

In [6]:
%cd ..

/content


In [7]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

 99% 147M/149M [00:09<00:00, 18.9MB/s]
100% 149M/149M [00:09<00:00, 15.8MB/s]


In [8]:
!unzip -qq brain-tumor-mri-dataset.zip

In [11]:
train_dataset_dir = Path("Training") # in "train", put a folder for each class, with folder name = class name
valid_dataset_dir = Path("Testing")   # in "val", same as above

In [12]:
# These are settings for ensuring input images to DinoV2 are properly sized

class ResizeAndPad:
    def __init__(self, target_size, multiple):
        self.target_size = target_size
        self.multiple = multiple

    def __call__(self, img):
        # Resize the image
        img = transforms.Resize(self.target_size)(img)

        # Calculate padding
        pad_width = (self.multiple - img.width % self.multiple) % self.multiple
        pad_height = (self.multiple - img.height % self.multiple) % self.multiple

        # Apply padding
        img = transforms.Pad((pad_width // 2, pad_height // 2, pad_width - pad_width // 2, pad_height - pad_height // 2))(img)

        return img

image_dimension = 256

# This is what DinoV2 sees
target_size = (image_dimension, image_dimension)

# Below are functions that every image will be passed through, including data augmentations
data_transforms = {
    "train": transforms.Compose(
        [
            ResizeAndPad(target_size, 14),
            transforms.RandomRotation(360),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

# Here is where we wrap up our images, which are in folders (defined above) where the folder name is class name
# As long as you defined the "train" folder above with sub-folders for each class, below will "just work"
image_datasets = {
    "train": datasets.ImageFolder(train_dataset_dir, data_transforms["train"])
}

dataloaders = {
    "train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=8, shuffle=True)
}

class_names = image_datasets["train"].classes

In [13]:
class_names

['glioma', 'meningioma', 'notumor', 'pituitary']

In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
device

device(type='cuda')

In [16]:
!wget https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth

--2024-03-19 20:27:59--  https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.102, 108.157.254.15, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88291785 (84M) [binary/octet-stream]
Saving to: ‘dinov2_vits14_reg4_pretrain.pth’

dinov2_vits14_reg4_ 100%[===================>]  84.20M   275MB/s    in 0.3s    

2024-03-19 20:28:00 (275 MB/s) - ‘dinov2_vits14_reg4_pretrain.pth’ saved [88291785/88291785]



In [18]:
# Define a new classifier layer that contains a few linear layers with a ReLU to make predictions positive
class DinoVisionTransformerClassifier(nn.Module):

    def __init__(self, model_size="small"):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.model_size = model_size

        # loading a model with registers
        n_register_tokens = 4

        if model_size == "small":
            model = vit_small(patch_size=14,
                              img_size=526,
                              init_values=1.0,
                              num_register_tokens=n_register_tokens,
                              block_chunks=0)
            self.embedding_size = 384
            self.number_of_heads = 6

        elif model_size == "base":
            model = vit_base(patch_size=14,
                             img_size=526,
                             init_values=1.0,
                             num_register_tokens=n_register_tokens,
                             block_chunks=0)
            self.embedding_size = 768
            self.number_of_heads = 12

        elif model_size == "large":
            model = vit_large(patch_size=14,
                              img_size=526,
                              init_values=1.0,
                              num_register_tokens=n_register_tokens,
                              block_chunks=0)
            self.embedding_size = 1024
            self.number_of_heads = 16

        elif model_size == "giant":
            model = vit_giant2(patch_size=14,
                               img_size=526,
                               init_values=1.0,
                               num_register_tokens=n_register_tokens,
                               block_chunks=0)
            self.embedding_size = 1536
            self.number_of_heads = 24

        # Download pre-trained weights and place locally as-needed:
        # - small: https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth
        # - base:  https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_reg4_pretrain.pth
        # - large: https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_reg4_pretrain.pth
        # - giant: https://dl.fbaipublicfiles.com/dinov2/dinov2_vitg14/dinov2_vitg14_reg4_pretrain.pth
        model.load_state_dict(torch.load(Path('/content/dinov2_vits14_reg4_pretrain.pth')))

        self.transformer = deepcopy(model)

        self.classifier = nn.Sequential(nn.Linear(self.embedding_size, 256), nn.ReLU(), nn.Linear(256, 1))

    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x

model = DinoVisionTransformerClassifier("small")

In [19]:
model = model.to(device)
model = model.train()

In [20]:
# change the binary cross-entropy loss below to a different loss if using more than 2 classes
# https://pytorch.org/docs/stable/nn.html#loss-functions

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [21]:
num_epochs = 50

In [ ]:
def calculate_accuracy(outputs, labels):
    # Convert outputs to probabilities using sigmoid
    probabilities = torch.sigmoid(outputs)
    # Convert probabilities to predicted classes
    predicted_classes = probabilities > 0.5
    # Calculate accuracy
    correct_predictions = (predicted_classes == labels.byte()).sum().item()
    total_predictions = labels.size(0)
    return correct_predictions / total_predictions

epoch_losses = []
epoch_accuracies = []

print("Training...")
for epoch in range(num_epochs):
    batch_losses = []
    batch_accuracies = []

    for data in dataloaders["train"]:
        # get the input batch and the labels
        batch_of_images, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # model prediction
        output = model(batch_of_images.to(device)).squeeze(dim=1)

        # compute loss and do gradient descent
        loss = criterion(output, labels.float().to(device))

        loss.backward()
        optimizer.step()

        batch_losses.append(loss.item())

        # Calculate and record batch accuracy
        accuracy = calculate_accuracy(output, labels.to(device))
        batch_accuracies.append(accuracy)

    epoch_losses.append(np.mean(batch_losses))
    epoch_accuracy = np.mean(batch_accuracies)
    epoch_accuracies.append(epoch_accuracy)

    print("  -> Epoch {}: Loss = {:.5f}, Accuracy = {:.3f}%".format(epoch, epoch_losses[-1], 100*epoch_accuracy))

Training...
  -> Epoch 0: Loss = 23.66168, Accuracy = 32.300%
  -> Epoch 1: Loss = 22.82400, Accuracy = 40.616%
  -> Epoch 2: Loss = 22.62972, Accuracy = 42.665%
  -> Epoch 3: Loss = 22.51077, Accuracy = 43.452%


In [ ]:
local_directory = os.getcwd()

In [ ]:
# Plotting accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epoch_accuracies, label='Accuracy', color='blue')
plt.title("Training Accuracy")
plt.xlabel("Epoch Number")
plt.ylabel("Accuracy")

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(epoch_losses, label='Loss', color='red')
plt.title("Training Loss")
plt.xlabel("Epoch Number")
plt.ylabel("Loss")

In [ ]:
# During inference / testing / deployment, we want to remove data augmentations from the input transform:
inference_preprocessing = transforms.Compose([ ResizeAndPad(target_size, 14),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                             ]
                                            )

In [ ]:
def inference_on_validation_data(inference_model, n_test_images=1):

    for class_name in class_names:
        # gather N validation images per class
        val_class_dir = "{}/{}".format(valid_dataset_dir, class_name)
        val_test_images = os.listdir(val_class_dir)[:n_test_images]

        for val_test_image in val_test_images:

            if ".jpg" not in val_test_image and ".png" not in val_test_image:
                continue

            # load image
            img = Image.open("{}/{}".format(val_class_dir, val_test_image))

            # pre-process image and load onto device
            img_tensor = inference_preprocessing(img)
            img_tensor = img_tensor.unsqueeze(0)
            input_tensor = img_tensor.to(device)

            # run model on input image data
            with torch.no_grad():
                embeddings = inference_model.transformer(input_tensor)
                x = inference_model.transformer.norm(embeddings)
                output_tensor = inference_model.classifier(x)

                # sigmoid
                score = output_tensor[0][0].item()
                score = 1 / (1 + np.exp(-score))

                # for binary classification, we can just interpret class name based on the 1 output score
                predicted_class = class_names[1] if score > 0.5 else class_names[0]

                print("\n\n\nDinoV2 Predicted = {} vs. Ground Truth = {}:".format(predicted_class, class_name))
                plt.imshow(img)
                plt.show()

In [ ]:
# save the model
torch.save(model.state_dict(), '{}/classifier.pth'.format(local_directory))

In [ ]:
# reloading the model with trained classification weights to demonstrate deployment
classifier = DinoVisionTransformerClassifier("small")

classifier.load_state_dict(torch.load('{}/classifier.pth'.format(local_directory)))

classifier = classifier.to(device)
classifier.eval()

In [ ]:
inference_on_validation_data(inference_model=classifier, n_test_images=1)